# Master notebook

In [1]:
VERSION = '0.0'

In [2]:
from __future__ import absolute_import, division, print_function 
import os
import re
import sys
import time

sys.path.append(os.path.join('.', '..')) 
import utils
import utils_DL
import utils_s160159 as u_s

import matplotlib.pyplot as plt
import numpy as np
# import sklearn.datasets
import tensorflow as tf

from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import confusion_matrix

## Load data

In [3]:
data_dir = './../Data_1'
logs_path = './logs'
# NUM_SUBJECTS = len([ff for ff in os.listdir(data_dir) if '_img_fpz' in ff])
NUM_SUBJECTS = 5 # 20
NUM_CLASSES = 6

# TRAIN_IDX = list(range(NUM_SUBJECTS))
# VAL_IDX = list(range(NUM_SUBJECTS))
VAL_PER = 0.05
VAL_TRAIN_ID = NUM_SUBJECTS-int(NUM_SUBJECTS * VAL_PER)-1


In [4]:
#Load all subjects into memory
subjects_list = []
for i in range(1,NUM_SUBJECTS+1):
    print("Loading subject %d..." %(i))
    inputs_night1, targets_night1, _  = u_s.load_spectrograms(data_path=data_dir, subject_id=i, night_id=1)
    if(i!=20):
        inputs_night2, targets_night2, _  = u_s.load_spectrograms(data_path=data_dir, subject_id=i, night_id=2)
    else:
        inputs_night2 = np.empty((0,224,224,3),dtype='uint8')
        targets_night2 = np.empty((0,NUM_CLASSES),dtype='uint8')           

    current_inputs = np.concatenate((inputs_night1,inputs_night2),axis=0)
    current_targets = np.concatenate((targets_night1, targets_night2),axis=0)
    
    subjects_list.append([current_inputs, current_targets])
    
    #mean_images = np.zeros((5,224,224,3))
    #mean_images[0,] = np.mean(current_inputs[current_targets==0,], axis=0)
    #mean_images[1,] = np.mean(current_inputs[current_targets==1,], axis=0)
    #mean_images[2,] = np.mean(current_inputs[current_targets==2,], axis=0)
    #mean_images[3,] = np.mean(current_inputs[np.logical_or(current_targets==3, current_targets==4),], axis=0)
    #mean_images[4,] = np.mean(current_inputs[current_targets==5,], axis=0)
    #subjects_list.append([current_inputs, current_targets, mean_images])
    
    
# extract image shapes
IMAGE_SHAPE = subjects_list[0][0].shape

Loading subject 1...
Loading subject 2...
Loading subject 3...
Loading subject 4...
Loading subject 5...


## Building the model

In [5]:
# hyperameters
HEIGTH, WIDTH, NCHANNELS = IMAGE_SHAPE[1], IMAGE_SHAPE[2], IMAGE_SHAPE[3]
PADDING = 'same'

#global_step = tf.Variable(0, trainable=False, name='global_step_traning')

#MINI_BATCHES = tf.Variable(250, trainable=False, name='size_of_mini_batch')
#DECAY_STEPS = tf.Variable(250, trainable=False, name='number_of_decay_steps')
# L_RATE_START = tf.Variable(10e-5, trainable=False, name='learning_rate_start', dtype='float32')
# BETA_1 = tf.Variable(0.9, trainable=False, name='learning_rate_first_moment') # first moment
# BETA_2 = tf.Variable(0.999, trainable=False, name='learning_rate_second_moment') # second moment

# L_RATE = tf.Variable((tf.train.exponential_decay(learning_rate=L_RATE_START,
#                            global_step=global_step,
#                            decay_rate=BETA_1,
#                            decay_steps=DECAY_STEPS,
#                            staircase=True) + 
#           tf.train.exponential_decay(learning_rate=L_RATE_START,
#                            global_step=global_step,
#                            decay_rate=BETA_2,
#                            decay_steps=DECAY_STEPS,
#                            staircase=True)),
#                      trainable=False, name='learning_rate')
# L_RATE = tf.cast(L_RATE, dtype='float32_ref', name='learning_rate')


L_RATE = 10e-5
EPS = 1e-8
TRAIN_FEATURES = False





In [69]:
# https://www.cs.toronto.edu/~frossard/vgg16/vgg16.py

# Load the weights into memory
weights_dict = np.load(data_dir + '/' + 'vgg16_weights.npz', encoding='bytes')

def tf_conv2d(inputs, name, trainable=TRAIN_FEATURES):
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(shape=weights_dict[name + '_W'].shape, 
                                 initializer=tf.constant_initializer(weights_dict[name + '_W']), 
                                 name=scope + 'weights', 
                                 trainable=trainable)
        
        conv = tf.nn.conv2d(inputs, kernel, strides=[1, 1, 1, 1], padding='SAME')
        
        biases = tf.get_variable(shape=weights_dict[name + '_b'].shape,
                                 initializer=tf.constant_initializer(weights_dict[name + '_b']), 
                                 trainable=trainable, name=scope + 'biases')

        return(tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope))
        

def tf_max_pooling2d(inputs, name):
    with tf.name_scope(name) as scope:
        return(tf.nn.max_pool(inputs,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name=scope))        

def tf_fully_con(inputs, name, shape=[None, 4096]):
    with tf.name_scope(name) as scope:    
        fc_W = tf.get_variable(shape=shape, 
                               initializer=tf.contrib.layers.xavier_initializer(dtype=tf.float32,
                                                                                seed=22),
                               name=scope + 'weights', trainable=True)

        fc_b = tf.get_variable(shape=shape[1], 
                               initializer=tf.random_normal_initializer(mean=0, stddev=0.01, dtype=tf.float32),
                               trainable=True, name=scope + 'biases')
        
        in_flat = tf.reshape(inputs, [-1, shape[0]])
        return(tf.nn.relu(tf.nn.bias_add(tf.matmul(in_flat, fc_W), fc_b)))
        

In [70]:
# init model
tf.reset_default_graph()

# init placeholders
x_pl = tf.placeholder(tf.float32, [None, HEIGTH, WIDTH, NCHANNELS], name='input_placeholder')
y_pl = tf.placeholder(tf.float32, [None, NUM_CLASSES], name='target_placeholder')


print('Trace of the tensors shape as it is propagated through the network.')
print('Layer name \t Output size')
print('--------------------------------------------')

with tf.variable_scope('VVG16_layer'):
    print('x_pl \t\t', x_pl.get_shape())
    # level one
    conv1_1 = tf_conv2d(inputs=x_pl, name='conv1_1')
    conv1_2 = tf_conv2d(inputs=conv1_1, name='conv1_2')
    pool1 = tf_max_pooling2d(inputs=conv1_2, name='pool1')
    print('conv1_1 \t', conv1_1.get_shape())
    print('conv1_2 \t', conv1_2.get_shape())
    print('pool1 \t\t', pool1.get_shape())
    print('--------------------------------------------')
    
    # level two
    conv2_1 = tf_conv2d(inputs=pool1, name='conv2_1')
    conv2_2 = tf_conv2d(inputs=conv2_1, name='conv2_2')
    pool2 = tf_max_pooling2d(inputs=conv2_2, name='pool2')
    print('conv2_1 \t', conv2_1.get_shape())
    print('conv2_2 \t', conv2_2.get_shape())
    print('pool2 \t\t', pool2.get_shape())
    print('--------------------------------------------')
    
    # level three
    conv3_1 = tf_conv2d(inputs=pool2, name='conv3_1')
    conv3_2 = tf_conv2d(inputs=conv3_1, name='conv3_2')
    conv3_3 = tf_conv2d(inputs=conv3_2, name='conv3_3')
    pool3 = tf_max_pooling2d(inputs=conv3_3, name='pool_3')
    print('conv3_1 \t', conv3_1.get_shape())
    print('conv3_2 \t', conv3_2.get_shape())
    print('conv3_3 \t', conv3_3.get_shape())
    print('pool3 \t\t', pool3.get_shape())
    print('--------------------------------------------')
    
    # level four
    conv4_1 = tf_conv2d(inputs=pool3, name='conv4_1')
    conv4_2 = tf_conv2d(inputs=conv4_1, name='conv4_2')
    conv4_3 = tf_conv2d(inputs=conv4_2, name='conv4_3')
    pool4 = tf_max_pooling2d(inputs=conv4_3, name='pool_4')
    print('conv4_1 \t', conv4_1.get_shape())
    print('conv4_2 \t', conv4_2.get_shape())
    print('conv4_3 \t', conv4_3.get_shape())
    print('pool4 \t\t', pool4.get_shape())
    print('--------------------------------------------')

    # level five
    conv5_1 = tf_conv2d(inputs=pool4, name='conv5_1')
    conv5_2 = tf_conv2d(inputs=conv5_1, name='conv5_2')
    conv5_3 = tf_conv2d(inputs=conv5_2, name='conv5_3')
    pool5 = tf_max_pooling2d(inputs=conv5_3, name='pool_5')
    print('conv5_1 \t', conv5_1.get_shape())
    print('conv5_2 \t', conv5_2.get_shape())
    print('conv5_3 \t', conv5_3.get_shape())
    print('pool5 \t\t', pool5.get_shape())
    print('--------------------------------------------')

    # level six
    fc6 = tf_fully_con(inputs=pool5, name='fc6', shape=[int(np.prod(pool5.get_shape()[1:])), 4096])
    fc6_dropout = tf.layers.dropout(inputs=fc6, name='fc6_dropout', rate=0.5)

    # level seven
    fc7 = tf_fully_con(inputs=fc6_dropout, name='fc7', shape=[4096, 4096])
    fc7_dropout = tf.layers.dropout(inputs=fc7, name='fc7_dropout', rate=0.5)

    # level eigth
    fc8 = tf_fully_con(inputs=fc7_dropout, name='fc7', shape=[4096, NUM_CLASSES])
    #fc8 = tf_fully_con(inputs=fc7_dropout, name='fc7', shape=[4096, 1000])
    
    print('fc6 \t', fc6.get_shape())
    print('fc7 \t', fc7.get_shape())
    print('fc8 \t', fc8.get_shape()) 
    print('--------------------------------------------')


    
with tf.variable_scope('output_layer'):    
    l_out = tf.nn.softmax(fc8, name='l_out')
    print('out \t', l_out.get_shape())
    print('--------------------------------------------')
    
print('Model consits of ', utils_DL.num_params(), 'trainable parameters.')
## print all the variable names and shapes
#for var in tf.global_variables ():
#    s = var.name + " "*(40-len(var.name))
#    print(s, var.value().get_shape())

Trace of the tensors shape as it is propagated through the network.
Layer name 	 Output size
--------------------------------------------
x_pl 		 (?, 224, 224, 3)
conv1_1 	 (?, 224, 224, 64)
conv1_2 	 (?, 224, 224, 64)
pool1 		 (?, 112, 112, 64)
--------------------------------------------
conv2_1 	 (?, 112, 112, 128)
conv2_2 	 (?, 112, 112, 128)
pool2 		 (?, 56, 56, 128)
--------------------------------------------
conv3_1 	 (?, 56, 56, 256)
conv3_2 	 (?, 56, 56, 256)
conv3_3 	 (?, 56, 56, 256)
pool3 		 (?, 28, 28, 256)
--------------------------------------------
conv4_1 	 (?, 28, 28, 512)
conv4_2 	 (?, 28, 28, 512)
conv4_3 	 (?, 28, 28, 512)
pool4 		 (?, 14, 14, 512)
--------------------------------------------
conv5_1 	 (?, 14, 14, 512)
conv5_2 	 (?, 14, 14, 512)
conv5_3 	 (?, 14, 14, 512)
pool5 		 (?, 7, 7, 512)
--------------------------------------------
fc6 	 (?, 4096)
fc7 	 (?, 4096)
fc8 	 (?, 6)
--------------------------------------------
out 	 (?, 6)
-----------------------

### OPTIMISATION

In [71]:
with tf.variable_scope('loss_function'):
    # computing cross entropy per sample
    cross_entropy = -tf.reduce_sum(y_pl * tf.log(l_out + EPS), axis=[1])
    # averaging over samples
    cross_entropy = tf.reduce_mean(cross_entropy)

with tf.variable_scope('training'):
    # defining our optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=L_RATE)
    # applying the gradients
    #train_op = optimizer.minimize(cross_entropy, global_step=global_step)
    train_op = optimizer.minimize(cross_entropy)
    
with tf.variable_scope('performance'):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    prediction = tf.one_hot(tf.argmax(l_out, axis=1), depth=NUM_CLASSES)
    correct_prediction = tf.equal(tf.argmax(l_out, axis=1), tf.argmax(y_pl, axis=1))
    # averaging the one-hot encoded vector
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Test flow for model

In [72]:
## Launch TensorBoard, and visualize the TF graph
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.20)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    writer = tf.summary.FileWriter(logs_path, sess.graph)
    # close session
    sess.close()
# run in terminal
# """
# python -m webbrowser "http://localhost:6006/";
# tensorboard --logdir='./logs'
# """

In [73]:
# Test the forward pass    
x_batch = subjects_list[0][0][0:2]

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    y_pred = sess.run(fetches=l_out, feed_dict={x_pl: x_batch})

assert y_pred.shape == np.zeros((len(x_batch),NUM_CLASSES)).shape, "ERROR the output shape is not as expected!" \
        + " Output shape should be " + str(l_out.shape) + ' but was ' + str(y_pred.shape)

print('Forward pass successful!')

# close session
sess.close()

Forward pass successful!


## Train the model

In [74]:
# Training Loop
MAX_EPOCHS = 2
VAL_EPOCHS = 10
BATCH_SIZE = 50



# reset counter 
fetches_train = [train_op, cross_entropy, accuracy]
fetches_valid = [cross_entropy, accuracy]
fetches_test = [cross_entropy, accuracy, prediction]

capture_dict = {}

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    print('Begin training loop... \n')
    try:
        loo = LeaveOneOut()
        fold = 1
        for idx_tmp, idx_test in loo.split(range(NUM_SUBJECTS)):
            capture_dict[fold] = {}
            
            #
            valid_loss, valid_accuracy = [], []
            train_loss, train_accuracy = [], []
            test_loss, test_accuracy = [], []
            
            #
            print("Fold num %d\tSubject id %d" %(fold, idx_test + 1))
            #f = open('outputs/sleep5_fold'+str(fold), 'w').close()
            
            idx_train = idx_tmp[0:VAL_TRAIN_ID]
            idx_val = idx_tmp[VAL_TRAIN_ID:(NUM_SUBJECTS - 1)]

            # INTO TRAIN 
            train_data = [subjects_list[i] for i in idx_train]
            inputs_train = np.empty((0,224,224,3),dtype='uint8')  
            targets_train = np.empty((0,NUM_CLASSES),dtype='uint8') 
            for item in train_data:
                inputs_train = np.concatenate((inputs_train, item[0]),axis=0)
                targets_train = np.concatenate((targets_train, item[1]),axis=0)
            
            # INTO VALIDATION
            val_data = [subjects_list[i] for i in idx_val]
            inputs_val = np.empty((0,224,224,3),dtype='uint8')  
            targets_val = np.empty((0,NUM_CLASSES),dtype='uint8')
            for item in val_data:
                inputs_val = np.concatenate((inputs_val, item[0]),axis=0)
                targets_val = np.concatenate((targets_val, item[1]),axis=0)

            # INTO TEST
            test_data = [subjects_list[i] for i in idx_test]
            inputs_test = np.empty((0,224,224,3), dtype='uint8')  
            targets_test = np.empty((0,NUM_CLASSES), dtype='uint8')
            for item in test_data:
                inputs_test = np.concatenate((inputs_test,item[0]), axis=0)
                targets_test = np.concatenate((targets_test,item[1]), axis=0) 
                
            # Loop epochs
            for epoch in range(MAX_EPOCHS):
                print('\tEpoch: ' + str(epoch + 1) + ' of ' + str(MAX_EPOCHS))
                # loop mini batches
                _train_loss, _train_accuracy = [], []
                for x_batch, y_batch in utils.iterate_minibatches(batchsize=BATCH_SIZE, 
                                                                  inputs=inputs_train, 
                                                                  targets=targets_train, shuffle=False):
   
                    feed_dict_train = {x_pl: x_batch, 
                                       y_pl: y_batch}
                    _, _loss, _acc = sess.run(fetches_train, feed_dict_train)
                    #print("\t\tEp: %d\t L: %f\t ACCs: %f" %(epoch + 1, _loss, _acc))
                    
                    
                    
                    # append to epoch
                    _train_loss.append(_loss)
                    _train_accuracy.append(_acc)
                    
                    break
                #
                print("\t\tEp: %d\t L: %f\t ACCs: %f" %(epoch + 1, 
                                                        np.mean(_train_loss), 
                                                        np.mean(_train_accuracy)))
                
                
                
                # Compute validation loss and accuracy
                if epoch % VAL_EPOCHS == 0:
                    print('here')
                    # append mean
                    train_loss.append(np.mean(_train_loss))
                    train_accuracy.append(np.mean(_train_accuracy))
                    
                    #
                    for x_batch, y_batch in utils.iterate_minibatches(batchsize=BATCH_SIZE, 
                                                                      inputs=inputs_val, 
                                                                      targets=targets_val, shuffle=False):
                        #
                        feed_dict_valid = {x_pl: x_batch, y_pl: y_batch}
                        _loss, _acc = sess.run(fetches_valid, feed_dict_valid)
                        # append mean
                        valid_loss.append(_loss)
                        valid_accuracy.append(_acc)
                    
                    #
                    print("\t\tEpoch {} : Train Loss {:6.3f}, Train acc {:6.3f},  Valid loss {:6.3f},  Valid acc {:6.3f}".format(epoch, train_loss[-1], train_accuracy[-1], valid_loss[-1], valid_accuracy[-1]))
                
            # Loop test
            print('\tEvaluate test performance')
            pred = np.empty((0,NUM_CLASSES),dtype='uint8') 
            for x_batch, y_batch in utils.iterate_minibatches(batchsize=BATCH_SIZE, 
                                                              inputs=inputs_test, 
                                                              targets=targets_test, shuffle=False):

                feed_dict_test = {x_pl: x_batch, y_pl: y_batch}
                _loss, _acc, _pred = sess.run(fetches_test, feed_dict_test)
            
                # append prediction
                pred = np.concatenate((pred, _pred),axis=0)
                test_loss.append(_loss)
                test_accuracy.append(_acc)
                print('\t\tTest Loss {:6.3f}, Test acc {:6.3f}'.format(np.mean(test_loss), 
                                                                   np.mean(test_accuracy)))
            
            # calculate performance
            cm = confusion_matrix(y_pred=np.where(pred ==1)[1], 
                                  y_true=np.where(targets_test ==1)[1], labels=list(range(NUM_CLASSES)))
            capture_dict[fold] = {'cm': cm, 
                                  'train_loss': np.mean(train_loss),
                                  'train_accuracy': np.mean(train_accuracy),
                                  'test_loss': np.mean(test_loss),
                                  'test_accuracy': np.mean(test_accuracy),
                                  'valid_loss': np.mean(valid_loss),
                                  'valid_accuracy': np.mean(valid_accuracy)} 
            
                
            # increase fold
            fold += 1
            print('\n... end training loop')
            break
    except KeyboardInterrupt:
        pass

Begin training loop... 

Fold num 1	Subject id 1
	Epoch: 1 of 2
		Ep: 1	 L: 2.049725	 ACCs: 0.360000
here
		Epoch 0 : Train Loss  2.050, Train acc  0.360,  Valid loss    nan,  Valid acc    nan
	Epoch: 2 of 2
		Ep: 2	 L: 6.263031	 ACCs: 0.660000
	Evaluate test performance
		Test Loss 16.947, Test acc  0.080
		Test Loss 17.500, Test acc  0.050
		Test Loss 17.807, Test acc  0.033
		Test Loss 17.868, Test acc  0.030


## Save model

In [77]:
# https://www.tensorflow.org/versions/r1.0/programmers_guide/variables
# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    save_path = tf.train.Saver(var_list=tf.trainable_variables()).save(sess, "./models/model.ckpt")
    print("Model saved in file: %s" % save_path)

FailedPreconditionError: Attempting to use uninitialized value VVG16_layer/VVG16_layer/conv1_1/biases
	 [[Node: save_2/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_2/Const_0_0, save_2/SaveV2/tensor_names, save_2/SaveV2/shape_and_slices, VVG16_layer/VVG16_layer/conv1_1/biases, VVG16_layer/VVG16_layer/conv1_1/weights, VVG16_layer/VVG16_layer/conv1_2/biases, VVG16_layer/VVG16_layer/conv1_2/weights, VVG16_layer/VVG16_layer/conv2_1/biases, VVG16_layer/VVG16_layer/conv2_1/weights, VVG16_layer/VVG16_lay